In [1]:
import modulos as md

lista=md.lista_de_monedas ()

for symbol in lista:
    try:
        data = md.obtiene_historial(symbol)
        data = md.estrategia(data)
        md.backtesting(data,symbol)
    except:
        pass


C:\Users\Luis Gomez Freites\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Falla leyendo...


c:\LUCHO\personal\repopersonal\luchocrypto\FENIX\modulos.py:273: UserWarning: Some prices are larger than initial cash value. Note that fractional trading is not supported. If you want to trade Bitcoin, increase initial cash, or trade μBTC or satoshis instead (GH-134).
  bt = Backtest(data, Fenix, cash=balance, commission=.002, exclusive_orders=True)


MANAUSDT - 16.12380642135247
Falla leyendo...
Falla leyendo...
Falla leyendo...
Falla leyendo...
Falla leyendo...
Falla leyendo...
Falla leyendo...
MAGICUSDT - 18.510655204441036
STXUSDT - 34.84600199364013
LQTYUSDT - 64.2593370660395
JOEUSDT - 28.1417007134487
Falla leyendo...
SUIUSDT - 39.510001317208435
Falla leyendo...
Falla leyendo...
